## Analyze A/B Test Results

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [2]:
df=pd.read_csv(r'C:\program alaa\excel 001\ab_data.csv')

In [3]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


b. Use the below cell to find the number of rows in the dataset.

In [4]:
df.shape

(294478, 5)

number of rows=294478

c. The number of unique users in the dataset.

In [5]:
df['user_id'].value_counts()

746755    2
722274    2
892271    2
736112    2
728134    2
         ..
700160    1
698113    1
704258    1
702211    1
630732    1
Name: user_id, Length: 290584, dtype: int64

number of unique users =290584

d. The proportion of users converted.

In [6]:
df_conv= df[df['converted']==1]

In [7]:
proportion_conv=(len(df_conv)/len(df))*100

In [8]:
proportion_conv

11.96591935560551

The proportion of users converted=11.96%

e. The number of times the `new_page` and `treatment` don't line up.

In [9]:
df_mismatch1=df[(df['landing_page']=='new_page')&(df['group']!='treatment')]

In [10]:
df_mismatch1

,user_id,timestamp,group,landing_page,converted
22,767017,2017-01-12 22:58:14.991443,control,new_page,0
240,733976,2017-01-11 15:11:16.407599,control,new_page,0
490,808613,2017-01-10 21:44:01.292755,control,new_page,0
846,637639,2017-01-11 23:09:52.682329,control,new_page,1
850,793580,2017-01-08 03:25:33.723712,control,new_page,1
...,...,...,...,...,...
293894,741581,2017-01-09 20:49:03.391764,control,new_page,0
293996,942612,2017-01-08 13:52:28.182648,control,new_page,0
294200,928506,2017-01-13 21:32:10.491309,control,new_page,0
294253,886135,2017-01-06 12:49:20.509403,control,new_page,0


In [11]:
df_mismatch2=df[(df['landing_page']=='old_page')&(df['group']=='treatment')]

In [12]:
df_mismatch2

,user_id,timestamp,group,landing_page,converted
308,857184,2017-01-20 07:34:59.832626,treatment,old_page,0
327,686623,2017-01-09 14:26:40.734775,treatment,old_page,0
357,856078,2017-01-12 12:29:30.354835,treatment,old_page,0
685,666385,2017-01-23 08:11:54.823806,treatment,old_page,0
713,748761,2017-01-10 15:47:44.445196,treatment,old_page,0
...,...,...,...,...,...
293773,688144,2017-01-16 20:34:50.450528,treatment,old_page,1
293817,876037,2017-01-17 16:15:08.957152,treatment,old_page,1
293917,738357,2017-01-05 15:37:55.729133,treatment,old_page,0
294014,813406,2017-01-09 06:25:33.223301,treatment,old_page,0


In [13]:
no_rows=len(df_mismatch1)+len(df_mismatch2)
no_rows

3893

there are 3856 rows where the landing_page=new_page dont line up with a group category treatment

f. Do any of the rows have missing values?

In [14]:
df.isnull().sum().sum()

0

the data has no nulls

`2.` For the rows where **treatment** is not aligned with **new_page** or **control** is not aligned with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to provide how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [15]:

df_mismatch3=df[(df.landing_page=='old_page')&(df.group!='control')]
df_mismatch4=df[((df.landing_page!='old_page')&(df.group=='control'))]


In [16]:
frames=[df_mismatch1,df_mismatch2,df_mismatch3,df_mismatch4]
mismatching=pd.concat(frames)

In [17]:
mismatching

,user_id,timestamp,group,landing_page,converted
22,767017,2017-01-12 22:58:14.991443,control,new_page,0
240,733976,2017-01-11 15:11:16.407599,control,new_page,0
490,808613,2017-01-10 21:44:01.292755,control,new_page,0
846,637639,2017-01-11 23:09:52.682329,control,new_page,1
850,793580,2017-01-08 03:25:33.723712,control,new_page,1
...,...,...,...,...,...
293894,741581,2017-01-09 20:49:03.391764,control,new_page,0
293996,942612,2017-01-08 13:52:28.182648,control,new_page,0
294200,928506,2017-01-13 21:32:10.491309,control,new_page,0
294253,886135,2017-01-06 12:49:20.509403,control,new_page,0


In [18]:
df2=pd.DataFrame()
df2=df

In [19]:
df2.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [20]:
df2.drop(df2[((df2['landing_page']=='new_page')&(df2['group']!='treatment'))|((df2['landing_page']=='old_page')&(df2['group']=='treatment'))|
           ((df2.landing_page=='old_page')&(df2.group!='control'))|((df2.landing_page!='old_page')&(df2.group=='control'))].index,inplace=True)

In [21]:
df2.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [22]:
df2.shape

(290585, 5)

In [23]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [24]:
df2['user_id'].value_counts()

773192    2
630836    1
634271    1
753093    1
751044    1
         ..
803549    1
809694    1
807647    1
895712    1
630732    1
Name: user_id, Length: 290584, dtype: int64

In [25]:
number of unique ids=290584

SyntaxError: invalid syntax (<ipython-input-25-87b821da69cf>, line 1)

b. There is one **user_id** repeated in **df2**.  What is it?

773192 

c. What is the row information for the repeat **user_id**? 

In [ ]:
df2[df2['user_id']==773192]

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [ ]:
df2.drop(df2[(df2['timestamp']=='2017-01-09 05:37:58.781806')].index,inplace=True)

In [ ]:
df2[df2['user_id']==773192]

`4.` Use **df2** in the below cells to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [ ]:
(df2.converted==1).mean()

b. Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
df2.query('group=="control"')['converted'].mean()

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
df2.query('group=="treatment"')['converted'].mean()

d. What is the probability that an individual received the new page?

In [ ]:
n=df2[df2['landing_page']=='new_page']

In [ ]:
len(n)/len(df2)

e. Consider your results from a. through d. above, and explain below whether you think there is sufficient evidence to say that the new treatment page leads to more conversions.

**Your answer goes here.** no sufficient evidence as the propability of a customer getting convernted is 0.119 while the propability of a customer to convernt given that the landing page the customer browsed is the new page is only 0.118 which is nearly the same propability a customer converts in general so there is no evidence that the new_page had an effect 

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

**Put your answer here.** null hypotheses : The new page is equal or less in its impact on conversion actions  𝑝𝑛𝑒𝑤 =< 𝑝𝑜𝑙𝑑
alternative hypotheses: the new page performs better than the old page in increasing conversions 𝑝𝑛𝑒𝑤 > 𝑝𝑜𝑙𝑑

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

a. What is the **convert rate** for $p_{new}$ under the null? 

In [ ]:
𝑝𝑛𝑒𝑤=(df2.converted==1).mean()

In [ ]:
pnew

b. What is the **convert rate** for $p_{old}$ under the null? <br><br>

In [ ]:
pold=pnew

In [ ]:
pold

c. What is $n_{new}$?

In [ ]:
new_page=df2[df2['landing_page']=='new_page']

In [ ]:
new_page.shape

In [ ]:
  𝑛𝑛𝑒𝑤 =145310

d. What is $n_{old}$?

In [ ]:
old_page=df2[df2['landing_page']=='old_page']

In [ ]:
old_page.shape

In [ ]:
nold=145274

In [ ]:
df2.groupby('user_id')['converted'].sum()

sample size =number of ones in df2 =290584

e. Simulate $n_{new}$ transactions with a convert rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [ ]:
sample_new_page=new_page.sample(29058)
means_new=[]
for _ in range(10000):
    bootsample=sample_new_page.sample(29058,replace=True)
    means_new.append(bootsample.converted.mean())

In [ ]:
plt.hist(means_new)

In [ ]:
y=np.array(means_new)
pnew=y.mean()

In [ ]:
pnew

In [ ]:
new_page_converted=[]
new_page_converted=new_page['converted']

f. Simulate $n_{old}$ transactions with a convert rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [ ]:
old_page_converted=[]
old_page_converted=old_page['converted']

In [ ]:
sample_old_page=old_page.sample(29058)
means_old=[]
for _ in range(10000):
    bootsample=sample_old_page.sample(29058,replace=True)
    means_old.append(bootsample.converted.mean())

In [ ]:
plt.hist(means_old)


In [ ]:
x=np.array(means_old)
pold=x.mean()

In [ ]:
(x>pnew).mean() # the p-value is large which means that the new page is no different from the old page which will be proven 
#later on when plotting the differnence

In [ ]:
plt.hist(means_old)
plt.axvline(x=pnew,color='red')

In [ ]:
pold

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [ ]:
pnew-pold

In [ ]:
p_actual=pnew-pold

h. Simulate 10,000 $p_{new}$ - $p_{old}$ values using this same process similarly to the one you calculated in parts **a. through g.** above.  Store all 10,000 values in a numpy array called **p_diffs**.

In [ ]:
sample_df2=df2.sample(29058)
p_diff=[]
for _ in range(10000):
    bootsample=sample_df2.sample(29058,replace=True)
    old=bootsample[bootsample['landing_page']=='old_page']['converted'].mean()
    new=bootsample[bootsample['landing_page']=='new_page']['converted'].mean()
    p_diff.append(new-old)

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
plt.hist(p_diff)

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
np.percentile(p_diff,2.5),np.percentile(p_diff,97.5)

In [ ]:
p_diff=np.array(p_diff)

In [ ]:
plt.hist(p_diff)
plt.axvline(x=p_actual,color='red')

In [ ]:
(p_diff>p_actual).mean()

k. In words, explain what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

**Put your answer here.** the computed value is called the p-value it is used to show how extreme is the produced mean with respect to a certain sample distribution that if this mean is extreme the p-value is very small typically(0.005 or 0.001 in some applications)which is a an evidence that this mean doesnt belong to this population and is belonging to an entirely different population on the other hand if the p-value is large it means that the mean being tested is not different from other means in the population (not extreme) which means that it may have come from the the same population its tested to prove that it is significantlly different from it but its is proved to not be significantly different 


in this case of old and new pages:
the large p-value calculated above(0.9381) indicates that the difference in conversion rates is not due to a significant difference between the new and the old pages 
which means that the null hypotheses is true that the pnew=< pold

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
import statsmodels.api as sm

convert_old =old_page[old_page['converted']==1]['converted'].sum()
convert_new=new_page[new_page['converted']==1]['converted'].sum()
n_old_shape =old_page.shape
n_new_shape =new_page.shape

In [ ]:
n_old_shape

In [ ]:
n_old=145274

In [ ]:
convert_old 

In [ ]:
n_new_shape

In [ ]:
n_new=145310

In [ ]:
convert_new

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

In [26]:
counts=[17264,17489] #order of the parameter differs according to the alternative hypotheses H1:pnew>pold

In [27]:
nobs=[145310,145274]

In [28]:
#performing one-sided t-test 
from statsmodels.stats.proportion import proportions_ztest
test_statistic,p_value=proportions_ztest(counts,nobs,alternative='larger')

In [29]:
test_statistic

-1.3109241984234394

In [30]:
p_value

0.9050583127590245

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

**Put your answer here.** for a confidence interval of 95% the crossponding z values at the boundries of the rejection region are 1.96,-1.96 ,since our z-score is -0.7436 so it lies with in the 95% accepted region which means that we failed to reject the null hypotheses 
the p-value here calculated with out sampling its large 0.457 which means we again fail to reject the null hypotheses 

the values obtained in this section totally agrees with the findings in j and k

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you acheived in the previous A/B test can also be acheived by performing regression.<br><br>

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

**Put your answer here.** linear regression

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives.  However, you first need to create a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [345]:
df2['intercept']=1


In [346]:
df2[['old_page','new_page']] = pd.get_dummies(df['landing_page'])

In [351]:
df2[['treatment','control']]=pd.get_dummies(df['group'])

In [352]:
df2.head()

,user_id,timestamp,group,landing_page,converted,intercept,new_page,treatment,control
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,1,1,1,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,1,1,1,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,1,0,0,1
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,1,0,0,1
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,1,1,1,0


In [348]:
df2=df2.drop('old_page',axis=1) #we remove the the old_page column and consider the old_page as our baseline variable

In [353]:
df2=df2.drop('control',axis=1)

In [354]:
df2['ab_page']=df2['treatment']

In [355]:
df2.head()

,user_id,timestamp,group,landing_page,converted,intercept,new_page,treatment,ab_page
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,1,1,1,1
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,1,1,1,1
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,1,0,0,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,1,0,0,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,1,1,1,1


In [356]:
df2=df2.drop('treatment',axis=1)

In [357]:
df2.head()

,user_id,timestamp,group,landing_page,converted,intercept,new_page,ab_page
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,1,1,1
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,1,1,1
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,1,0,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,1,0,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,1,1,1


c. Use **statsmodels** to import your regression model.  Instantiate the model, and fit the model using the two columns you created in part **b.** to predict whether or not an individual converts.

In [358]:
lm=sm.OLS(df2['converted'],df2[['intercept','new_page','ab_page']])
results=lm.fit()

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [359]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              converted   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.720
Date:                Sat, 12 Mar 2022   Prob (F-statistic):              0.190
Time:                        10:23:24   Log-Likelihood:                -85267.
No. Observations:              290585   AIC:                         1.705e+05
Df Residuals:                  290583   BIC:                         1.706e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.1188      0.001    139.570      0.000       0.117       0.120
new_page       0.0008      0.001      1.312      0.190      -0.000       0.002
ab_page        0.0008      0.001      1.312      0.190      -0.000       0.002
==============================================================================
Omnibus:                   125554.337   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           414318.532
Skew:                           2.345   Prob(JB):                         0.00
Kurtosis:                       6.497   Cond. No.                     3.67e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.69e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in the **Part II**?

**Put your answer here.** p-value associated with the ab_page=0.190 its different because its interperted as follows:
the basline variable is the old page  which means that  if the page is the old page we predict the mean of  the conversions  to be 0.118 while if the landing page is the new page we predict the mean of the conversions to be 0.0008 more than that of the old page   
the null and alternative are a bit different that each of the ab_page and the new_page values are a comparsion to that of the
baseline category (old_page)



note:
the model has a problem as it assumes that all the variables used for prediction are independent from each other while the ab_page and the new_page variable are obivously strongly corelated that all the treatment group members are showen the new_page only according to the experiment rules 

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

**Put your answer here.** maybe the individual gender has an effect that for example maybe our products interests women more than men or maybe age will have an effect that the page desgin is more liked by kids or elderlys 

its not a good idea to add more variables because when the complexity of the model increases it becomes harder to have a clear interpertation of the results although we can be exposed to variables that are not linearly related to the target variable or variables that are corelated one another which violates the independence constrications of the model

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives. You will need to read in the **countries.csv** dataset and merge together your datasets on the approporiate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [360]:
countries_df = pd.read_csv(r'C:\program alaa\excel 001/countries.csv')
df_new = countries_df.set_index('user_id').join(df2.set_index('user_id'), how='inner')

In [363]:
df_new.head()

,country,timestamp,group,landing_page,converted,intercept,new_page,ab_page
user_id,,,,,,,,
630000,US,2017-01-19 06:26:06.548941,treatment,new_page,0,1,0,0
630001,US,2017-01-16 03:16:42.560309,treatment,new_page,1,1,0,0
630002,US,2017-01-19 19:20:56.438330,control,old_page,0,1,1,1
630003,US,2017-01-12 10:09:31.510471,treatment,new_page,0,1,0,0
630004,US,2017-01-18 20:23:58.824994,treatment,new_page,0,1,0,0


In [366]:
df_new['country'].value_counts

<bound method IndexOpsMixin.value_counts of user_id
630000    US
630001    US
630002    US
630003    US
630004    US
          ..
945994    UK
945996    US
945997    US
945998    CA
945999    US
Name: country, Length: 290585, dtype: object>

In [368]:
### Create the necessary dummy variables
df_new[['US','UK','CA']] = pd.get_dummies(df_new['country'])

In [369]:
df_new=df_new.drop('US',axis=1)#let US be our baseline variable

In [370]:
df_new.head()

,country,timestamp,group,landing_page,converted,intercept,new_page,ab_page,UK,CA
user_id,,,,,,,,,,
630000,US,2017-01-19 06:26:06.548941,treatment,new_page,0,1,0,0,0,1
630001,US,2017-01-16 03:16:42.560309,treatment,new_page,1,1,0,0,0,1
630002,US,2017-01-19 19:20:56.438330,control,old_page,0,1,1,1,0,1
630003,US,2017-01-12 10:09:31.510471,treatment,new_page,0,1,0,0,0,1
630004,US,2017-01-18 20:23:58.824994,treatment,new_page,0,1,0,0,0,1


h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [371]:
### Fit Your Linear Model And Obtain the Results
lm=sm.OLS(df_new['converted'],df_new[['intercept','new_page','ab_page','UK','CA']])
results=lm.fit()

In [372]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              converted   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.640
Date:                Sat, 12 Mar 2022   Prob (F-statistic):              0.178
Time:                        11:18:32   Log-Likelihood:                -85266.
No. Observations:              290585   AIC:                         1.705e+05
Df Residuals:                  290581   BIC:                         1.706e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.1145      0.003     41.494      0.000       0.109       0.120
new_page       0.0008      0.001      1.308      0.191      -0.000       0.002
ab_page        0.0008      0.001      1.308      0.191      -0.000       0.002
UK             0.0053      0.003      1.784      0.074      -0.001       0.011
CA             0.0042      0.003      1.514      0.130      -0.001       0.010
==============================================================================
Omnibus:                   125552.050   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           414302.956
Skew:                           2.345   Prob(JB):                         0.00
Kurtosis:                       6.497   Cond. No.                     9.28e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.51e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

if the country is US we predict the mean of the conversions to be 0.1145 which is different from our perivious predictions with out considering the country that the individual lives in which sugessts that the country affects the conversion rate 
we perdict individuals living in the UK to have a conversion mean 0.0053 more than those in the US
we perdict individuals living in the CA to have a conversion mean 0.0042 more than those in the US

<a id='conclusions'></a>
## Conclusions

Congratulations on completing the project! 

### Gather Submission Materials

Once you are satisfied with the status of your Notebook, you should save it in a format that will make it easy for others to read. You can use the __File -> Download as -> HTML (.html)__ menu to save your notebook as an .html file. If you are working locally and get an error about "No module name", then open a terminal and try installing the missing module using `pip install <module_name>` (don't include the "<" or ">" or any words following a period in the module name).

You will submit both your original Notebook and an HTML or PDF copy of the Notebook for review. There is no need for you to include any data files with your submission. If you made reference to other websites, books, and other resources to help you in solving tasks in the project, make sure that you document them. It is recommended that you either add a "Resources" section in a Markdown cell at the end of the Notebook report, or you can include a `readme.txt` file documenting your sources.

### Submit the Project

When you're ready, click on the "Submit Project" button to go to the project submission page. You can submit your files as a .zip archive or you can link to a GitHub repository containing your project files. If you go with GitHub, note that your submission will be a snapshot of the linked repository at time of submission. It is recommended that you keep each project in a separate repository to avoid any potential confusion: if a reviewer gets multiple folders representing multiple projects, there might be confusion regarding what project is to be evaluated.

It can take us up to a week to grade the project, but in most cases it is much faster. You will get an email once your submission has been reviewed. If you are having any problems submitting your project or wish to check on the status of your submission, please email us at dataanalyst-project@udacity.com. In the meantime, you should feel free to continue on with your learning journey by beginning the next module in the program.